In [476]:
import math
import numpy as np

In [477]:
def Estep(X, mixture_weights, means, cov_mat, p, G):
    # multivariate gaussian PDF
    probs = np.zeros((X.shape[0], G)) # n x g shape
    for k, mu in enumerate(means): # exponent term
        probs[:,k] = np.sum((np.linalg.inv(cov_mat) @ (X - mu).T) * (X - mu).T, axis=0)
    probs = np.exp(probs * -0.5)
    probs /= math.sqrt(np.linalg.det(cov_mat) * (2 * math.pi) ** p) 

    probs *= mixture_weights

    return (probs / probs.sum(axis=1, keepdims=True)) + 1e-10 # normalizing

In [478]:
def Mstep(X, probs, p, G):
    mixture_weights = np.mean(probs, axis=0)
    means = np.zeros((G, p))
    cov_mat = np.zeros((p, p))
    for k in range(G):
        means[k] = np.sum((X.T * probs[:,k]).T, 0) / sum(probs[:,k])
        tmp = X - means[k]
        cov_mat_k = tmp.T @ np.diag(probs[:,k]) @ tmp
        cov_mat_k /= np.sum(probs[:,k])
        cov_mat += cov_mat_k
    return mixture_weights, means, cov_mat

In [479]:
faithful = open('faithful.dat')
temp = []
for row in faithful:
    temp.append(row.split()[1:])
temp = temp[1:]
for i in range(len(temp)):
    for j in range(len(temp[i])):
        temp[i][j] = float(temp[i][j])
faithful = np.array(temp)

In [480]:
## Testing G = 2
n = len(faithful)
X = faithful
p = 2
G = 2

p1 = 10/n
p2 = 1 - p1
mixture_weights = [p1, p2]

mu1 = X[:10].mean(0)
mu2 = X[10:].mean(0)
means = np.array([mu1, mu2])

cov_mat_L = sum([(X[l] - mu1).reshape((2, 1)) @ (X[l] - mu1).reshape((1, 2)) for l in range(10)])
cov_mat_R = sum([(X[l] - mu1).reshape((2, 1)) @ (X[l] - mu1).reshape((1, 2)) for l in range(10, len(X))])
cov_mat = (cov_mat_L + cov_mat_R) / n

In [481]:

for i in range(20):
    r = Estep(X, mixture_weights, means, cov_mat, p, G)
    mixture_weights, means, cov_mat = Mstep(X, r, p, G)
    print(mixture_weights)


[0.03602766 0.96397234]
[0.0352983 0.9647017]
[0.03511552 0.96488448]
[0.03507015 0.96492985]
[0.03505887 0.96494113]
[0.03505606 0.96494394]
[0.03505536 0.96494464]
[0.03505518 0.96494482]
[0.03505514 0.96494486]
[0.03505513 0.96494487]
[0.03505512 0.96494488]
[0.03505512 0.96494488]
[0.03505512 0.96494488]
[0.03505512 0.96494488]
[0.03505512 0.96494488]
[0.03505512 0.96494488]
[0.03505512 0.96494488]
[0.03505512 0.96494488]
[0.03505512 0.96494488]
[0.03505512 0.96494488]


## Part 2

In [19]:
#BW
def forward(t,i,x, mz,w,A,B,alphas):
    if alphas[t,i] != 0:
        return alphas[t,i]
    elif t == 0:#changed from 1
        return w[i]*B[i,x[0]]
    else:
        s=0
        for j in range(mz):
            s+=forward(t-1,j,x, mz,w,A,B,alphas)*A[j,i]*B[i,x[t]]
        return s
        

In [20]:
def backward(t,i,x, mx, mz,w,A,B,betas):
    n = len(x)
    if betas[t,i] !=0:
        return betas[t,i]
    elif t == n-1:#changed from n
        return 1
    else:
        s = 0
        for j in range(mz):
            s+=A[i,j]*B[j,x[t+1]]*backward(t+1,j,x, mx, mz,w,A,B,betas)#changed bxt
        return s

In [21]:
def biggamma(t,i,j,x, mx,mz,w,A,B,alphas,betas):
    return forward(t,i,x, mz,w,A,B,alphas)*A[i,j]*B[j,x[t+1]]*backward(t+1,j,x, mx, mz,w,A,B,betas)

In [22]:
def BW_onestep(data, mx, mz,w,A,B):
    #Estep 
    n = len(data)
    T = len(data)
    #calculate the alphas and betas
    oldA = A.copy()
    alphas = np.zeros((n,mz))
    for t in range(n):
        for i in range(mz):
            alphas[t,i] = forward(t,i,data, mz,w,A,B, alphas)
    betas = np.zeros((n, mz))
    for t in range(n-1,-1,-1):
        for j in range(mz):
            betas[t,j] = backward(t,j,data,mx, mz,w,A,B,betas)

    gamma = np.zeros((mz,mz,T-1))
    for i in range(mz):
        for j in range(mz):
            for t in range(T-1):
                gamma[i][j][t] = biggamma(t,i,j,data, mx, mz,w,A,B,alphas,betas)
    
    for i in range(mz):
        den = np.sum(gamma[i,:,:])
        for j in range(mz):
            num = np.sum(gamma[i,j,:])
            A[i][j] = num/den
    
    margiGamma = np.zeros((T,mz))
    for t in range(T):
        for i in range(mz):
            if t==0:#changed from 1
                s = 0
                for j in range(mz):
                    s+=gamma[i,j,t]#changed from 1
                margiGamma[t,i] = s
            elif t == T-1:#changed from len(x)
                s = 0
                for j in range(mz):
                    s+=gamma[j,i,t-1]
                margiGamma[t,i] = s
            else:
                s = 0
                for j in range(mz):
                    #print(t,i,j)
                    s+=gamma[i,j,t]
                margiGamma[t,i] = s
    for i in range(mz):
        den = 0
        for t in range(T):
            den += margiGamma[t,i]
        for l in range(mx):
            num = 0
            for t in range(T):
                if data[t] == l:
                    num+=margiGamma[t,i]
            B[i,l] = num/den
            #print(num,den)
            
    return(A,B)

In [23]:
#test data
part2 = open('coding4_part2_data.txt')
temp = []
for row in part2:
    temp.append(row.split())
for i in range(len(temp)):
    temp[i] = int(temp[i][0])-1#changed because it was one indexed
part2 = np.array(temp)
w = np.array([.5,.5])
A = np.array([[.5,.5],
              [.5,.5]])
B = np.array([[1/9,3/9,5/9],
              [1/6,2/6,3/6]])
mz = 2
mx=3

In [24]:
for i in range(100):
    #print("i: ", i)
    A,B = BW_onestep(part2, mx, mz,w,A,B)
print(A)
print(B)

[[0.49793938 0.50206062]
 [0.44883431 0.55116569]]
[[0.22159897 0.20266127 0.57573976]
 [0.34175148 0.17866665 0.47958186]]
